In [19]:
import re
import pandas as pd
from datetime import datetime

df = pd.read_excel('caffe.xlsx')

# Adjusting quantity

In [20]:
def adjust_quantity(desc, qty, discount):
    pattern = r"(\d+)\+(\d+)"
    match = re.search(pattern, desc)
    if match:
        x, y = map(int, match.groups())
        qty = qty * x + y
        desc = re.sub(pattern, "", desc).strip()
        discount = match.group(0)
    return desc, qty, discount


def extract_quantity(desc, qty):
    pattern = r"^\d+(?! SEASONS)"
    match = re.search(pattern, desc)
    if match:
        x = int(match.group(0))
        qty = qty * x
        desc = re.sub(pattern, "", desc).strip()
    return desc, qty

In [21]:
df["Discount Type"] = ""

df[["Item Desc", "Quantity", "Discount Type"]] = df.apply(
    lambda row: pd.Series(adjust_quantity(row["Item Desc"], row["Quantity"], row["Discount Type"])),
    axis=1
)

df[["Item Desc", "Quantity"]] = df.apply(
    lambda row: pd.Series(extract_quantity(row["Item Desc"], row["Quantity"])),
    axis=1
)

# Add weekdays

In [22]:
def add_day_of_week(date):
    wd = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    return wd[date.weekday()]

In [23]:
df["Week day"] = df["Date"].apply(add_day_of_week)

# Add weeknumbers

In [24]:
df["Week number"] = df["Date"].dt.isocalendar().week
df.to_csv('caffe_change.csv', index=False)

# Add Hour column

In [ ]:
df["Hour"] = pd.to_datetime(df["Time"].astype(str), errors="coerce").dt.hour

/var/folders/n1/rhpqtbg90rq_85vnsylnfq2r0000gn/T/ipykernel_36887/2284766911.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Hour"] = pd.to_datetime(df["Time"].astype(str), errors="coerce").dt.hour


# Split btl and glss

In [26]:
df['GLS/BTL'] = df['Item Desc'].apply(
    lambda x: 'GLS' if any(w in x.upper() for w in ['GLS', 'GLASS']) 
    else ('BTL' if any(w in x.upper() for w in ['BTL', 'BOTTLE']) else '')
)

In [27]:
df["Bottle/Glass"] = df["GLS/BTL"].map({"GLS": "Glass", "BTL": "Bottle"}).fillna("")
df.to_csv('caffe_change.csv', index=False)

# Add TOPPINGS categorie

In [28]:
def change_to_toppings(desc, category):
    pattern = r'\b(ADD?)\b'
    match = re.search(pattern, desc)
    if match:
        desc = re.sub(pattern, "", desc).strip()
        category = "ADD-ONS"
    return desc, category


df[["Item Desc", "Category"]] = df.apply(
    lambda row: pd.Series(change_to_toppings(row["Item Desc"], row["Category"])),
    axis=1
)


# Write to new csv

In [29]:
df.to_excel('caffe_change.xlsx', index=False)